In [ ]:
import pickle
import os.path
import pathlib

from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = [
            'https://www.googleapis.com/auth/gmail.send',
            'https://www.googleapis.com/auth/gmail.readonly',
         ]


def Service():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                pathlib.Path('~/credentials/gmail_api_creds.json').expanduser(),
                SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)
    return service

    # Call the Gmail API
    results = service.users().labels().list(userId='me').execute()
    labels = results.get('labels', [])

    if not labels:
        print('No labels found.')
    else:
        print('Labels:')
        for label in labels:
            print(label['name'])

In [ ]:
# slow
service = Service()

In [ ]:
from email.mime.text import MIMEText
import base64

In [ ]:
def create_message(sender, to, cc, subject, message_text):
    message = MIMEText(message_text)
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject
    raw_message = base64.urlsafe_b64encode(message.as_string().encode("utf-8"))
    return {
        'raw': raw_message.decode("utf-8")
    }


def create_draft(service, user_id, message_body):
    try:
        message = {'message': message_body}
        draft = service.users().drafts().create(userId=user_id, body=message).execute()

        print("Draft id: %s\nDraft message: %s" % (draft['id'], draft['message']))

        return draft
    except Exception as e:
        print('An error occurred: %s' % e)
        return None  

In [ ]:
import mimetypes
from email.mime.base import MIMEBase
from email.mime.application import MIMEApplication

In [ ]:
from email.mime.multipart import MIMEMultipart

def send_message(service, user_id, message):
    try:
        message = service.users().messages().send(userId=user_id, body=message).execute()

        print('Message Id: %s' % message['id'])

        return message
    except Exception as e:
        print('An error occurred: %s' % e)
        return None


def create_message_with_attachment(sender, to, subject, message_text, file):
    message = MIMEMultipart()
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject

    msg = MIMEText(message_text)
    message.attach(msg)

    content_type, encoding = mimetypes.guess_type(file)

    if content_type is None or encoding is not None:
        content_type = 'application/octet-stream'

    main_type, sub_type = content_type.split('/', 1)
    print(f"main_type: {main_type}")
    print(f"sub_type: {sub_type}")

    if main_type == 'text':
        fp = open(file, 'rb')
        msg = MIMEText(fp.read().decode("utf-8"), _subtype=sub_type)
        fp.close()

    elif main_type == 'application' and sub_type == 'pdf':   
        temp = open(file, 'rb')
        msg = MIMEApplication(temp.read(), _subtype=sub_type)
        temp.close()
    elif main_type == 'image':
        fp = open(file, 'rb')
        msg = MIMEImage(fp.read(), _subtype=sub_type)
        fp.close()
    elif main_type == 'audio':
        fp = open(file, 'rb')
        msg = MIMEAudio(fp.read(), _subtype=sub_type)
        fp.close()
    else:
        fp = open(file, 'rb')
        msg = MIMEBase(main_type, sub_type)
        msg.set_payload(fp.read())
        fp.close()
    filename = os.path.basename(file)
    msg.add_header('Content-Disposition', 'attachment', filename=filename)
    message.attach(msg)

    ## Part 4 encode the message (the message should be in bytes)
    message_as_bytes = message.as_bytes() # the message should converted from string to bytes.
    message_as_base64 = base64.urlsafe_b64encode(message_as_bytes) #encode in base64 (printable letters coding)
    raw = message_as_base64.decode()  # need to JSON serializable (no idea what does it means)
    return {'raw': raw} 

In [ ]:
file="/home/david/Downloads/send_to_pb/[2010.05365] ArXiving Before Submission Helps Everyone.pdf"

In [ ]:
msg = create_message_with_attachment(sender='David',
                              to='dzlob@pbsync.com',
                              subject='SEND_TO_PB',
                              message_text="# THIS MESSAGE WAS AUTOGENERATED BY SEND_TO_PB",
                              file="/home/david/Downloads/send_to_pb/[2010.05365] ArXiving Before Submission Helps Everyone.pdf"
                              )
msg['raw'][:30]

main_type: application
sub_type: pdf


'Q29udGVudC1UeXBlOiBtdWx0aXBhcn'

In [ ]:
send_message(service, 'david.dobrinskiy@gmail.com', message=msg)

Message Id: 1752ce8d5ee0876f


{'id': '1752ce8d5ee0876f',
 'threadId': '1752ce8d5ee0876f',
 'labelIds': ['SENT']}